In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
import pandas as pd
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import keras_tuner as kt
from sklearn.utils import shuffle

In [2]:
pip install tensorflow

  Using cached tensorflow-2.16.1-cp312-cp312-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.3.2-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached Markdown-3.6-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (


[notice] A new release of pip is available: 23.3.2 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install keras-tuner

  Using cached keras_tuner-1.4.7-py3-none-any.whl.metadata (5.4 kB)
  Using cached kt_legacy-1.0.5-py3-none-any.whl.metadata (221 bytes)
Using cached keras_tuner-1.4.7-py3-none-any.whl (129 kB)
Using cached kt_legacy-1.0.5-py3-none-any.whl (9.6 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Mounted at /content/drive


In [7]:
df= pd.read_csv('train.csv')

In [8]:
df.head()

,name,x1,x2,y1,y2,classname
0,2756.png,69,126,294,392,face_with_mask
1,2756.png,505,10,723,283,face_with_mask
2,2756.png,75,252,264,390,mask_colorful
3,2756.png,521,136,711,277,mask_colorful
4,6098.jpg,360,85,728,653,face_no_mask


In [9]:
#drop null and duplicate values
df=df.drop_duplicates()
df=df.dropna(axis=0)

In [10]:
df=df.drop(labels=['x1','x2','y1','y2'], axis=1)


In [11]:
df.head()

,name,classname
0,2756.png,face_with_mask
1,2756.png,face_with_mask
2,2756.png,mask_colorful
3,2756.png,mask_colorful
4,6098.jpg,face_no_mask


In [12]:
unique_classnames = df['classname'].unique()

# Print the unique class names
print(unique_classnames)

['face_with_mask' 'mask_colorful' 'face_no_mask'
 'face_with_mask_incorrect' 'mask_surgical' 'face_other_covering'
 'scarf_bandana' 'eyeglasses' 'helmet' 'face_shield' 'sunglasses' 'hood'
 'hat' 'goggles' 'hair_net' 'hijab_niqab' 'other' 'gas_mask'
 'balaclava_ski_mask' 'turban']


In [13]:
for classname in unique_classnames:
    count = df[df['classname'] == classname].shape[0]
    print(f"Class: {classname}, Count: {count}")

Class: face_with_mask, Count: 4180
Class: mask_colorful, Count: 1876
Class: face_no_mask, Count: 1568
Class: face_with_mask_incorrect, Count: 150
Class: mask_surgical, Count: 2430
Class: face_other_covering, Count: 1372
Class: scarf_bandana, Count: 260
Class: eyeglasses, Count: 914
Class: helmet, Count: 187
Class: face_shield, Count: 160
Class: sunglasses, Count: 358
Class: hood, Count: 159
Class: hat, Count: 823
Class: goggles, Count: 192
Class: hair_net, Count: 287
Class: hijab_niqab, Count: 173
Class: other, Count: 39
Class: gas_mask, Count: 55
Class: balaclava_ski_mask, Count: 134
Class: turban, Count: 94


In [14]:
#class mapping


# Assuming your dataset is in a DataFrame called `df` with a column 'class'
class_mapping = {
    'face_with_mask': 'face_with_mask',
    'mask_colorful': 'face_with_mask',
    'balaclava_ski_mask': 'face_with_mask',
    'gas_mask': 'face_with_mask',
    'mask_surgical': 'face_with_mask',
    'face_no_mask': 'face_no_mask',
    'face_with_mask_incorrect': 'face_no_mask',
    'face_other_covering': 'face_with_mask',
    'scarf_bandana': 'face_no_mask',
    'eyeglasses': 'face_no_mask',
    'helmet': 'face_no_mask',
    'face_shield': 'face_no_mask',
    'sunglasses': 'face_no_mask',
    'hood': 'face_no_mask',
    'hat': 'face_no_mask',
    'goggles': 'face_no_mask',
    'hair_net': 'face_no_mask',
    'hijab_niqab': 'face_with_mask',
    'other': 'face_no_mask',
    'turban': 'face_no_mask'
}

# Create a new column 'new_class' based on the mapping
df['class'] = df['classname'].map(class_mapping)

# Optionally, you can drop the old 'class' column if it's no longer needed
# df = df.drop(columns=['class'])

# Print the new class distribution
print(df['class'].value_counts())


class
face_with_mask    10220
face_no_mask       5191
Name: count, dtype: int64


In [15]:
df.head()

,name,classname,class
0,2756.png,face_with_mask,face_with_mask
1,2756.png,face_with_mask,face_with_mask
2,2756.png,mask_colorful,face_with_mask
3,2756.png,mask_colorful,face_with_mask
4,6098.jpg,face_no_mask,face_no_mask


In [ ]:
#since binary classification has to be implemented i have changed everywhere on the dataset to detect any type of mask as a mask: be it a colorful mask or any other type of mask apart from a surgical mask

In [16]:

df = df.drop_duplicates()
df=df.dropna().reset_index(drop=True)

In [17]:
df.head()

,name,classname,class
0,2756.png,face_with_mask,face_with_mask
1,2756.png,mask_colorful,face_with_mask
2,6098.jpg,face_no_mask,face_no_mask
3,6427.png,face_with_mask_incorrect,face_no_mask
4,6427.png,mask_surgical,face_with_mask


In [18]:
df.shape

(10818, 3)

In [22]:
import pandas as pd
from sklearn.utils import resample

# Assuming your dataframe is named df and has a column 'class' indicating the classes

# Separate the two classes
df_face_with_mask = df[df['class'] == 'face_with_mask']
df_face_no_mask = df[df['class'] == 'face_no_mask']

# Downsample the majority class
df_face_with_mask_downsampled = resample(df_face_with_mask, 
                                         replace=False,    # sample without replacement
                                         n_samples=3931,   # to match minority class
                                         random_state=42)  # for reproducibility

# Combine the downsampled majority class with the minority class
df_balanced = pd.concat([df_face_with_mask_downsampled, df_face_no_mask])

# Shuffle the dataframe to mix the classes
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the balanced dataframe
print(df_balanced['class'].value_counts())


class
face_with_mask    3931
face_no_mask      3931
Name: count, dtype: int64


In [26]:
df=df_balanced

In [62]:
# Function to load and resize an image
def load_and_resize_image(image_path, target_size=(128, 128)):
    with Image.open(image_path) as img:
        # If any dimension is larger than the target size, resize the image
        if img.width != target_size[0] or img.height != target_size[1]:
            img = img.resize(target_size,Image.BICUBIC)
        img_array = np.array(img)
        if len(img_array.shape) == 2:
            img_array = np.expand_dims(img_array, axis=-1)  # Add channel dimension if grayscale
        if img_array.shape[2] == 1:
            img_array = np.concatenate([img_array]*3, axis=-1)  # Convert grayscale to RGB
        return img_array

# Directory containing the images
image_dir = 'images'

# List to hold the resized images
resized_images = []

for name in df['name']:
    img_path = os.path.join(image_dir, name)
    if os.path.exists(img_path):
        resized_image = load_and_resize_image(img_path)
        resized_images.append(resized_image)
    else:
        resized_images.append(None)  # Append None if image not found
df['photo'] = resized_images


DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngI

In [28]:
df.head()


,name,classname,class,photo
0,3799.png,face_with_mask,face_with_mask,"[[[37, 62, 95], [54, 94, 139], [67, 133, 199],..."
1,2010.jpg,mask_surgical,face_with_mask,"[[[31, 28, 23], [31, 28, 23], [30, 27, 22], [2..."
2,4543.png,face_with_mask,face_with_mask,"[[[86, 84, 78], [118, 118, 112], [141, 140, 13..."
3,3514.png,mask_colorful,face_with_mask,"[[[64, 57, 49], [67, 60, 51], [70, 63, 55], [7..."
4,5863.jpg,face_no_mask,face_no_mask,"[[[152, 120, 82], [154, 121, 84], [155, 121, 8..."


In [29]:
X = np.array(df['photo'].tolist(), dtype=np.float32)
z = np.array(df['classname'].tolist(), dtype=str)
y=  [0 if label == 'face_no_mask' else 1 for label in z]
k= np.array(y)
num_classes = 2
y = to_categorical(k, num_classes)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes for debugging
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (6289, 128, 128, 3)
X_test shape: (1573, 128, 128, 3)
y_train shape: (6289, 2)
y_test shape: (1573, 2)


In [30]:
X_train = X_train / 225.0
X_test = X_test / 225.0


In [31]:
# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Assume X_train and y_train are your training data and labels
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_test, y_test, batch_size=32)


I have two groups now :
1) Data with normalised images for feeding in the model
2) The labels which I will use for binary classification
0: for a face without a face mask
1: face with a face mask

Start to train my model VGG16 Fine tuned

In [32]:
#BUILDING THE VGG MODEL
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))




In [33]:
for layer in vgg16.layers:
    layer.trainable = False

In [34]:
vgg16.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [35]:
def build_model(hp):
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    for layer in vgg16.layers:
        layer.trainable = False

    flatten = Flatten()(vgg16.output)

    dense_1 = Dense(units=hp.Int('units_1', min_value=100, max_value=500, step=100), activation='relu')(flatten)
    dropout_1 = Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1))(dense_1)

    dense_2 = Dense(units=hp.Int('units_2', min_value=100, max_value=500, step=100), activation='relu')(dropout_1)
    dropout_2 = Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1))(dense_2)

    dense_3 = Dense(units=hp.Int('units_3', min_value=100, max_value=500, step=100), activation='relu')(dropout_2)
    dropout_3 = Dropout(rate=hp.Float('dropout_3', min_value=0.2, max_value=0.5, step=0.1))(dense_3)

    dense_4 = Dense(units=hp.Int('units_4', min_value=100, max_value=500, step=100), activation='relu')(dropout_3)
    dropout_4 = Dropout(rate=hp.Float('dropout_4', min_value=0.2, max_value=0.5, step=0.1))(dense_4)

    dense_output = Dense(units=2, activation='softmax')(dropout_4)

    model = Model(inputs=vgg16.input, outputs=dense_output)
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='LOG')),
        metrics=['accuracy']
    )

    return model


In [42]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='vgg16_tuning'
)

tuner.search_space_summary()

# Use your data
# X_train, y_train, X_test, y_test should be defined here

tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.
Optimal number of units in the first dense layer: {best_hps.get('units_1')}
Optimal dropout rate for the first dropout layer: {best_hps.get('dropout_1')}
Optimal learning rate: {best_hps.get('learning_rate')}
""")


Trial 5 Complete [00h 37m 07s]
val_accuracy: 0.891290545463562

Best val_accuracy So Far: 0.891290545463562
Total elapsed time: 2d 18h 20m 27s

The hyperparameter search is complete.
Optimal number of units in the first dense layer: 200
Optimal dropout rate for the first dropout layer: 0.30000000000000004
Optimal learning rate: 8.685494403599878e-05



In [37]:
model = build_model(best_hps)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │     4,096,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        50,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 200)            │        20,20

 Total params: 18,982,990 (72.41 MB)

 Trainable params: 4,268,302 (16.28 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [43]:
model = tuner.hypermodel.build(best_hps)
import logging
logging.basicConfig(level=logging.DEBUG)
epochs = 20
batch_size = 10

try:
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        validation_data=(X_test, y_test),
        verbose=1
    )
    print("Training completed successfully.")
    print("History keys:", history.history.keys())
except Exception as e:
    logging.exception("Error during model fitting")


Epoch 1/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 459s 2s/step - accuracy: 0.8508 - loss: 0.4155 - val_accuracy: 0.8837 - val_loss: 0.3414
Epoch 2/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 444s 2s/step - accuracy: 0.8756 - loss: 0.3491 - val_accuracy: 0.8837 - val_loss: 0.3274
Epoch 3/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 433s 2s/step - accuracy: 0.8666 - loss: 0.3343 - val_accuracy: 0.8862 - val_loss: 0.3187
Epoch 4/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 442s 2s/step - accuracy: 0.8821 - loss: 0.2858 - val_accuracy: 0.8900 - val_loss: 0.3128
Epoch 5/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 448s 2s/step - accuracy: 0.8919 - loss: 0.2688 - val_accuracy: 0.8900 - val_loss: 0.3541
Epoch 6/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 443s 2s/step - accuracy: 0.9126 - loss: 0.2220 - val_accuracy: 0.8779 - val_loss: 0.3272
Epoch 7/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 438s 2s/step - accuracy: 0.9175 - loss: 0.2132 - val_accuracy: 0.8779 - val_loss: 0.3700
Epoch 8/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 442s 2s/step - accuracy: 0.9307 - loss: 0.1834 - val_accu

In [77]:
model.save('my_model.h5')